In [ ]:
import os 
import matplotlib.pyplot as plt
plt.style.use(['science', 'ieee', 'no-latex'])
import numpy as np
import pandas as pd
import cv2
import re
import PIL
from IPython.display import Image, display
from scipy import spatial

In [ ]:
def mkdir(save_dir):
    if os.path.exists(save_dir):
        for i in os.listdir(save_dir):
            path_file = os.path.join(save_dir, i)

            if os.path.isfile(path_file):
                os.remove(path_file)

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

In [ ]:
gen_dir = "ImageNet/generated_inputs/"
cam_dir_normal = "ImageNet/Grad_CAM_output/normal/"
cam_dir_random = "ImageNet/Grad_CAM_output/random/"

save_dir_normal = 'ImageNet/Experiment_cos_sim/normal/'
mkdir(save_dir_normal)
save_dir_random = 'ImageNet/Experiment_cos_sim/random/'
mkdir(save_dir_random)


In [ ]:
# input_dir = "ImageNet/Grad_CAM_output/xxxxx/"
def get_img_sim(cam_input_dir, fuzz_status):
    cos_sim_dict = {}

    _cam_perturb_lv_list = os.listdir(cam_input_dir)
    for _cam_perturb_lv in _cam_perturb_lv_list:
        _cam_dir_list = os.listdir(cam_input_dir + _cam_perturb_lv)
        if(_cam_dir_list != []):
            for _imgs_dir in _cam_dir_list:

                gen_img = None
                perturb_img = None
                
                _imgs_dir_split = _imgs_dir.split('_')
                _img_id = _imgs_dir_split[0]
                
                if(fuzz_status == 'normal'):
                    # get gen_filenames (adversarial samples)
                    normal_gen_filenames = next(os.walk(gen_dir + _cam_perturb_lv + '/gen_img/'), (None, None, []))[2]
                    # get gen_img (adversarial samples)
                    for file in normal_gen_filenames:
                        _img_path = file.split('_')
                        _origin_img_id = _img_path[0]  # 0000000xxx

                        if(_origin_img_id == _img_id):
                            gen_img = '/gen_img/' + file  # gen_img = "/gen_img/00000xxx_label1_label2_stamp.png"
                            break

                    # perturbation filenames
                    normal_perturb_filenames = next(os.walk(gen_dir + _cam_perturb_lv + '/perturb/'), (None, None, []))[2]
                    # perturbation image 
                    for file in normal_perturb_filenames:
                        _img_path = file.split('_')
                        _origin_img_id = _img_path[0]  # 0000000xxx

                        if(_origin_img_id == _img_id):
                            perturb_img = '/perturb/' + file
                            break

                    
                elif(fuzz_status == 'random'):
                    # get gen_filenames (adversarial samples)
                    random_gen_filenames = next(os.walk(gen_dir + _cam_perturb_lv + '/random_img/'), (None, None, []))[2]
                    # get gen_img (adversarial samples)
                    for file in random_gen_filenames:
                        _img_path = file.split('_')
                        _origin_img_id = _img_path[0]  # 0000000xxx

                        if(_origin_img_id == _img_id):
                            gen_img = '/random_img/' + file
                            break

                    # perturbation filenames
                    random_perturb_filenames = next(os.walk(gen_dir + _cam_perturb_lv + '/random_perturb/'), (None, None, []))[2]
                    # perturbation image 
                    for file in random_perturb_filenames:
                        _img_path = file.split('_')
                        _origin_img_id = _img_path[0]  # 0000000xxx

                        if(_origin_img_id == _img_id):
                            perturb_img = '/random_perturb/' + file
                            break

                # reading images
                Image1 = cv2.imread('ImageNet/seeds_20/' + 'ILSVRC2012_test_' + _img_id + '.JPEG')
                Image2 = cv2.imread(gen_dir + _cam_perturb_lv + gen_img)
                Image3 = cv2.imread(gen_dir + _cam_perturb_lv + perturb_img)

                RGB_img1 = cv2.cvtColor(Image1, cv2.COLOR_BGR2RGB)
                RGB_img2 = cv2.cvtColor(Image2, cv2.COLOR_BGR2RGB)
                RGB_img3 = cv2.cvtColor(Image3, cv2.COLOR_BGR2RGB)

                # Adds a subplot at the 1st position
                #fig.add_subplot(rows, columns, 1)
                # showing image
                #plt.imshow(RGB_img1)
                #plt.axis('off')
                #plt.title(_img_id + " (Original)")

                # Adds a subplot at the 2nd position
                #fig.add_subplot(rows, columns, 2)
                # showing image
                #plt.imshow(RGB_img2)
                #plt.axis('off')
                #plt.title(_img_id + " (DLFuzz Generated)")

                # Adds a subplot at the 3rd position
                #fig.add_subplot(rows, columns, 3)
                # showing image
                #plt.imshow(RGB_img1)
                #plt.axis('off')
                #plt.title(_img_id + " (Perturbation)")

                cams = next(os.walk(cam_input_dir + _cam_perturb_lv + '/' + _imgs_dir), (None, None, []))[2]
                cams_original = next(os.walk('ImageNet/Grad_CAM_output_originals/ILSVRC2012_test_' + _img_id), (None, None, []))[2]

                cos_sim_list = []

                for idx in range(0, 26):
                    # only work on heatmap
                    if(idx % 2 == 1):
                        # calculate similarity
                        img1 = PIL.Image.open('ImageNet/Grad_CAM_output_originals/ILSVRC2012_test_' + _img_id + '/' + cams_original[idx])
                        img2 = PIL.Image.open(cam_input_dir + _cam_perturb_lv + '/' + _imgs_dir + '/' + cams[idx])

                        # get layer name
                        _img_name = cams_original[idx].split('_')
                        _layer_name = _img_name[3] + " " + _img_name[4]

                        img1 = img1.resize((round(img2.size[0]), round(img2.size[1])))
                        img2 = img2.resize((round(img2.size[0]), round(img2.size[1])))

                        cam_array1 = np.array(img1)
                        cam_array2 = np.array(img2)

                        cam_array1 = cam_array1.flatten()
                        cam_array2 = cam_array2.flatten()

                        cam_array1 = cam_array1/255
                        cam_array2 = cam_array2/255

                        similarity = -1 * (spatial.distance.cosine(cam_array1, cam_array2) - 1)
                        cos_sim_list.append(similarity)

                        # create figure (subfigure)



                # dict["path"] = plt_sim_list
                cos_sim_dict[cam_input_dir + _cam_perturb_lv + '/' + _imgs_dir] = cos_sim_list

    return cos_sim_dict
    

In [ ]:
def get_random_img(cam_input_dir):
    ...

In [ ]:
# input_folder = "ImageNet/Grad_CAM_output/xxxxx/"
def calculate_cos_sim(cam_input_dir):

    _input_dir_split = cam_input_dir.split('/')
    _fuzz_status = _input_dir_split[-2]
    # get sim value, return dict
    cos_sim_dict = get_img_sim(cam_input_dir, _fuzz_status)

    # print(cos_sim_dict)
    # return a dict
    return cos_sim_dict

In [ ]:
def data_process_layer(_input):
    layer_data = {"Block1_Conv1": [], "Block1_Conv2": [], "Block2_Conv1": [], "Block2_Conv2": [], "Block3_Conv1": [], "Block3_Conv2": [], "Block3_Conv3": [], "Block4_Conv1": [], "Block4_Conv2": [], "Block4_Conv3": [], "Block5_Conv1": [], "Block5_Conv2":[], "Block5_Conv3":[]}
    
    # counter
    idx_count = 0
    for layer in layer_data:
        for element in _input:
            if(idx_count >= 13):
                break
            else:
                # _layer_data_list is a list of all cos values of the image
                _layer_data_list = _input[element]
                layer_data[layer].append(_layer_data_list[idx_count])

        idx_count += 1

    # print(layer_data)
    
    return layer_data
    

In [ ]:
def data_process_perturb(_input):
    perturb_data = {"perturb_level_0_25":{}, "perturb_level_0_5": {}, "perturb_level_1":{}, "perturb_level_2":{}, "perturb_level_4":{}}
    
    # couter
    for element in _input:
        _element_split = element.split('/')
        _perturb_lv = _element_split[3]

        perturb_data[_perturb_lv][element] = _input[element]
    
    return perturb_data

In [ ]:
def draw_boxplot(input, fuzz_status):
    dataframe = pd.DataFrame(input)
    #print(dataframe)

    columns = get_dataframe_column(dataframe)

    fig, ax = plt.subplots()
    ax.boxplot(columns)
    plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], 
    ['1-1', '1-2', '2-1', '2-2', '3-1', '3-2', '3-3', '4-1', '4-2', '4-3', '5-1', '5-2', '5-3'], 
    rotation=45)
    
    #if(fuzz_status == "normal"):
    #    ax.set_title("cosine similarity on adversial samples\' layers")
    #elif(fuzz_status == "random"):
    #    ax.set_title("cosine similarity on\n random noise perturbation samples\' layers")

    ax.set_xlabel("layers")
    ax.set_ylabel("cos similarity")
    plt.show()

    fig.savefig("boxplot_" + fuzz_status + '.pdf')
    

In [ ]:
def draw_plot(_input, ax):
    if(_input.empty):
        return
    else:
        columns = get_dataframe_column(_input)
        ax.plot(columns)
        plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
        ['1-1', '1-2', '2-1', '2-2', '3-1', '3-2', '3-3', '4-1', '4-2', '4-3', '5-1', '5-2', '5-3'], 
        rotation=45)
        
    

In [ ]:
def draw_line_plot(_input, fuzz_status):
    fig, ax = plt.subplots()
    ax.set_xlabel("layers")
    ax.set_ylabel("cos similarity")

    # input: nested dict
    perturb_lv_0_25 = data_process_layer(_input['perturb_level_0_25'])
    perturb_lv_0_5 = data_process_layer(_input["perturb_level_0_5"])
    perturb_lv_1 = data_process_layer(_input["perturb_level_1"])
    perturb_lv_2 = data_process_layer(_input["perturb_level_2"])
    perturb_lv_4 = data_process_layer(_input["perturb_level_4"])

    dataframe_0_25 = get_avg(perturb_lv_0_25)
    dataframe_0_5 = get_avg(perturb_lv_0_5)
    dataframe_1 = get_avg(perturb_lv_1)
    dataframe_2 = get_avg(perturb_lv_2)
    dataframe_4 = get_avg(perturb_lv_4)

    draw_plot(dataframe_0_25, ax)
    draw_plot(dataframe_0_5, ax)
    draw_plot(dataframe_1, ax)
    draw_plot(dataframe_2, ax)
    draw_plot(dataframe_4, ax)

    plt.show()

    fig.savefig("lineplot_" + fuzz_status + '.pdf')

In [ ]:
def get_avg(_input):
    # _input is a dict
    # return a dataframe
    for element in _input:
        if(_input[element] != []):
            data_array = _input[element]
            avg = []
            avg.append(sum(data_array) / len(data_array))
            _input.update({element: avg})
    #print(_input)

    dataframe = pd.DataFrame(_input)
    return dataframe
    

In [ ]:
def get_dataframe_column(dataframe):

    if(dataframe.empty):
        return None
    else:
        Block1_Conv1 = dataframe['Block1_Conv1']
        Block1_Conv2 = dataframe['Block1_Conv2']
        Block2_Conv1 = dataframe['Block2_Conv1']
        Block2_Conv2 = dataframe['Block2_Conv2']
        Block3_Conv1 = dataframe['Block3_Conv1']
        Block3_Conv2 = dataframe['Block3_Conv2']
        Block3_Conv3 = dataframe['Block3_Conv3']
        Block4_Conv1 = dataframe['Block4_Conv1']
        Block4_Conv2 = dataframe['Block4_Conv2']
        Block4_Conv3 = dataframe['Block4_Conv3']
        Block5_Conv1 = dataframe['Block5_Conv1']
        Block5_Conv2 = dataframe['Block5_Conv2']
        Block5_Conv3 = dataframe['Block5_Conv3']

        columns = [Block1_Conv1, Block1_Conv2, Block2_Conv1, Block2_Conv2, Block3_Conv1, Block3_Conv2, Block3_Conv3, Block4_Conv1, Block4_Conv2, Block4_Conv3, Block5_Conv1, Block5_Conv2, Block5_Conv3]
        return columns

In [ ]:
cos_sim_normal = calculate_cos_sim(cam_dir_normal)
layer_data_normal = data_process_layer(cos_sim_normal)
draw_boxplot(layer_data_normal, "normal")
perturb_data_normal = data_process_perturb(cos_sim_normal)
draw_line_plot(perturb_data_normal, "normal")


cos_sim_random = calculate_cos_sim(cam_dir_random)
layer_data_random = data_process_layer(cos_sim_random)
draw_boxplot(layer_data_random, "random")
perturb_data_random = data_process_perturb(cos_sim_random)
draw_line_plot(perturb_data_random, "random")